[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giordamaug/EG-identification---Data-Science-in-App-Springer/blob/main/notebook/EssentialGenes_ML.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/EG-identification---Data-Science-in-App-Springer/master/notebook/EssentialGenes_ML.ipynb)
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/giordamaug/EG-identification---Data-Science-in-App-Springer/blob/main/notebook/EssentialGenes_ML.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# Loading required libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)

# Download dataset from Github

In [2]:
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/bio_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/net_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/gtex_attributes.csv

--2022-05-03 21:25:19--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3966521 (3.8M) [text/plain]
Saving to: ‘ppi.csv’

ppi.csv             100%[===================>]   3.78M  --.-KB/s    in 0.09s   

2022-05-03 21:25:19 (39.9 MB/s) - ‘ppi.csv’ saved [3966521/3966521]

--2022-05-03 21:25:19--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

# Load the label (and encode)

In [3]:
labels = pd.read_csv("labels.csv", index_col='name')
from sklearn import preprocessing
from collections import Counter
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels['CS0_vs_CS6-9'].values)  
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(classes_mapping, Counter(y))

{'E': 0, 'NE': 1} Counter({1: 3069, 0: 745})


In [ ]:
import pandas as pd
from sklearn import preprocessing
node_name, label_name = 'name', 'label_CS_ACH_most_freq'
# read node attributes
df = pd.read_csv('/content/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv', 
                 sep='\t', index_col=node_name)   # load csv and set 'name' as index
x = df.select_dtypes(include=np.number)           # Attributes are numeric columns
labels = df.select_dtypes(exclude=np.number)      # labels are not numeric columns

# Load the PPI network
The PPI networks is loaded from a CSV file, where
*   `A` is the column name for edge source (gene name)
*   `B` is the column name for edge target (gene name)
*   `weight` is the column name for edge weight
Only some method use the PPI netoworks, as an example all GCN methods, and Node2Vec.



In [4]:
ppi = pd.read_csv('ppi.csv')                   # read ppi from CSV file

# Load attributes to be used
We identified three sets of attributes:
1. bio attributes, related to gene information (such as, expression, etc.)
2. net attributes, derived from role of gene/node in the network (such as, degree, centrality, etc.)
3. GTEX-* attribute, additional biological information of genes 

In this code snippets you choose:
+ the attributes to include, 
+ the normalization function for node attributes (Z-swcore or MinMax)

After loading, valuew of each attribute are corrected by filling NaN with the mean in the range, while Infinte value with the maximum in the range.


In [32]:
#@title Choose attributes { form-width: "20%" }
normalize_node = "zscore" #@param ["", "zscore", "minmax"]
bio_df = pd.read_csv("bio_attributes.csv", index_col='name')
gtex_df = pd.read_csv("gtex_attributes.csv", index_col='name')
net_df = pd.read_csv("net_attributes.csv", index_col='name')
x = pd.concat([bio_df, gtex_df, net_df], axis=1)
print(f'Found {x.isnull().sum().sum()} NaN values and {np.isinf(x).values.sum()} Infinite values')
for col in x.columns[x.isna().any()].tolist():
  mean_value=x[col].mean()          # Replace NaNs in column with the mean of values in the same column
  if mean_value is not np.nan:
    x[col].fillna(value=mean_value, inplace=True)
  else:                             # otherwise, if the mean is NaN, remove the column
    x = x.drop(col, 1)
if normalize_node == 'minmax':
  print("X attributes normalization (minmax)...")
  x = (x-x.min())/(x.max()-x.min())
elif normalize_node == 'zscore':
  print("X attributes normalization (zscore)...")
  x = (x-x.mean())/x.std()
x

Found 4009 NaN values and 0 Inf values
X attributes normalization (zscore)...


,gc_content,Gtex_kidney,gene_disease_ass_count,oncodb_expression,orth_count,gene_length,HPA_kidney,mf_coal,bp_coal,cc_coal,...,eigen,hub,trans,PR,triangles_numb,motif1,motif2,motif3,motif5,strength
name,,,,,,,,,,,,,,,,,,,,,
ENSG00000001036,-0.729174,-0.045984,0.968586,0.138709,0.841720,0.274147,0.032190,-0.786993,-2.010631e-01,0.092126,...,0.635709,0.635709,1.689553,-0.198491,-0.323159,-0.262870,-0.366587,-0.413753,-0.334908,-0.377416
ENSG00000001461,0.188372,-0.040266,-0.062302,-5.021751,0.804865,-0.428269,-0.112955,-0.786993,-6.562619e-01,-0.857818,...,-0.241634,-0.241634,-0.604062,-0.388473,-0.352559,1.871153,1.518046,2.046828,0.867106,2.974460
ENSG00000001561,1.294786,-0.038476,-0.234117,0.979769,0.743441,0.328474,-0.096788,-0.517857,-5.045290e-01,-0.145360,...,-0.241682,-0.241682,-0.341207,-0.374996,-0.352430,0.571236,-0.039539,-0.289543,-0.207759,-0.320851
ENSG00000001630,1.039558,-0.047117,-0.405931,0.293986,0.817150,-0.323292,-0.025095,0.827822,-1.251967e-01,-0.382846,...,1.776662,1.776662,0.295472,0.939715,3.648337,-0.382918,-0.206605,-0.036322,-0.049724,0.770654
ENSG00000001631,1.749092,-0.040969,0.023605,-0.985283,0.804865,-0.376388,-0.111900,0.020415,-4.933021e-02,-0.145360,...,-0.231378,-0.231378,-0.915414,-0.224040,-0.345494,-0.356584,-0.388387,-0.292171,-0.337920,-0.060934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288283,0.506131,-0.046534,-0.062302,1.388574,-1.136152,0.442719,-0.158290,0.558687,-6.562619e-01,-0.620332,...,-0.241729,-0.241729,-0.852707,-0.395741,-0.350489,-0.433221,-0.458901,-0.411040,-0.345450,-0.369035
ENSG00000288359,-0.253173,-0.047135,-0.405931,1.177928,-0.165643,4.149431,-0.160750,-0.248721,-1.251967e-01,-0.620332,...,-0.241801,-0.241801,-1.524051,-0.399182,-0.352585,0.980115,0.458642,0.744192,0.289282,0.047911
ENSG00000288407,-0.768734,-0.045608,0.710864,0.800208,-0.042794,2.695912,-0.162156,-1.056129,-2.021489e-15,-1.095304,...,-0.241748,-0.241748,-1.190409,-0.389895,-0.347642,-0.255933,-0.349041,-0.432552,-0.337944,-0.461633


# k-fold cross validation with: SVM, RF, XGB, MLP, RUS

In [33]:
#@title Choose classifier { run: "auto", form-width: "20%" }
method = "XGB" #@param ["SVM", "XGB", "RF", "MLP", "RUS", "DUMMY"]
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neural_network import MLPClassifier
import sys
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from sklearn.dummy import DummyClassifier
seed = 123
set_seed(seed)
nfolds = 5
kf = KFold(n_splits=nfolds)
accuracies, mccs = [], []
cm = [[0,0],[0,0]]
X = x.to_numpy()

if method == 'SVM':
  clf = SVC(random_state=seed, probability=True)
elif method == 'MLP':
  clf = MLPClassifier(random_state=seed)
elif method == 'RF':
  clf = RandomForestClassifier(random_state=seed)
elif method == 'XGB':
  clf = XGBClassifier(random_state=seed)
elif method == 'RUS':
  clf = RUSBoostClassifier(random_state=seed)
else:
  raise ValueError("Method not implemented!")

cma = np.array([[0,0],[0,0]])
mm = np.array([], dtype=np.int)
predictions = np.array([])
columns_names = ["Accuracy","BA", "Sensitivity", "Specificity","MCC", 'CM']
scores = pd.DataFrame(columns=columns_names)
for fold, (train_idx, test_idx) in enumerate(tqdm(kf.split(np.arange(len(X))), total=kf.get_n_splits(), desc=f"{nfolds}-fold")):
    train_x, train_y, test_x, test_y = X[train_idx], y[train_idx], X[test_idx], y[test_idx],
    mm = np.concatenate((mm, test_idx))
    probs = clf.fit(train_x, train_y).predict(test_x)
    preds = (probs > 0.5) * 1
    cm = confusion_matrix(test_y, preds)
    cma += cm
    predictions = np.concatenate((predictions, preds))
    scores = scores.append(pd.DataFrame([[accuracy_score(test_y, preds), balanced_accuracy_score(test_y, preds), 
        cm[0,0]/(cm[0,0]+cm[0,1]), cm[1,1]/(cm[1,0]+cm[1,1]), 
        matthews_corrcoef(test_y, preds), cm]], columns=columns_names, index=[fold]))
df_scores = pd.DataFrame(scores.mean(axis=0)).T
df_scores.index=[f'{method}']
df_scores['CM'] = [cma]
df_scores

5-fold: 100%|██████████| 5/5 [00:10<00:00,  2.10s/it]


,Accuracy,BA,Sensitivity,Specificity,MCC,CM
XGB,0.89198,0.787012,0.61748,0.956543,0.6331,"[[465, 280], [132, 2937]]"


In [34]:
p = np.zeros(len(y))
p[mm] = predictions
labels['predictions'] = ['NE' if x>0 else 'E' for x in p]
labels

,CS0_vs_CS6-9,predictions
name,,
ENSG00000001036,NE,NE
ENSG00000001461,NE,NE
ENSG00000001561,NE,NE
ENSG00000001630,NE,E
ENSG00000001631,NE,NE
...,...,...
ENSG00000288283,NE,NE
ENSG00000288359,NE,NE
ENSG00000288407,NE,NE
